## Mehrshad Saadatinia - 97243039

In [478]:
import pandas as pd
import sqlite3 as sql
import math

In [479]:
try:
    conn1 = sql.connect('database.sqlite')
    print(conn1)
except sql.Error as error:
    print('connection error occurred')

In [480]:
#viewing tables in each database
cursor1 = conn1.cursor() #football

### Question 1 : football analysis

In [491]:
#cursor1 : football database

#one
cursor1.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables_tups = cursor1.fetchall()
print("one) tables names: ",[t[0] for t in tables_tups])
print("\n-----------------------------------------------------------------------------------\n")

#two
country_df = pd.read_sql('SELECT name FROM Country', conn1)
print('two) list of countries: ', [c[0] for c in country_df.values.tolist()])
print("\n-----------------------------------------------------------------------------------\n")

#three
league_country_df = pd.read_sql('SELECT Country.name AS country_name, League.name AS league_name \
FROM League INNER JOIN Country ON League.country_id=Country.id', conn1)
print('three) list of countries & their leagues:\n', league_country_df) #drop ids
print("\n----------------------------------------------------------------------------------\n")

#four
teams_sorted_df = pd.read_sql('SELECT team_long_name FROM Team ORDER BY team_long_name ASC;', conn1)
#sorts alphanumerically , so if a name starts with number, goes first
print('four) list of teams sorted alphanumerically:')
for team in teams_sorted_df.values.tolist():
    print(team[0]) 


one) tables names:  ['sqlite_sequence', 'Player_Attributes', 'Player', 'Match', 'League', 'Country', 'Team', 'Team_Attributes']

-----------------------------------------------------------------------------------

two) list of countries:  ['Belgium', 'England', 'France', 'Germany', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Spain', 'Switzerland']

-----------------------------------------------------------------------------------

three) list of countries & their leagues:
    country_name               league_name
0       Belgium    Belgium Jupiler League
1       England    England Premier League
2        France            France Ligue 1
3       Germany     Germany 1. Bundesliga
4         Italy             Italy Serie A
5   Netherlands    Netherlands Eredivisie
6        Poland        Poland Ekstraklasa
7      Portugal  Portugal Liga ZON Sagres
8      Scotland   Scotland Premier League
9         Spain           Spain LIGA BBVA
10  Switzerland  Switzerland Super League

-

In [489]:
#five
spain_matches_df = pd.read_sql(
    """SELECT Country.name AS country_name, League.name AS league_name, 
    home_team_goal, away_team_goal, date, season
    FROM
      Country
         INNER JOIN League
            ON Country.id = League.country_id
            INNER JOIN Match ON League.id = Match.league_id
    WHERE Country.name='Spain'
    """
    , conn1)

print('\n five) list (dataframe) of matches in Spain along with other columns: \n')
spain_matches_df


 five) list (dataframe) of matches in Spain along with other columns: 



,country_name,league_name,home_team_goal,away_team_goal,date,season
0,Spain,Spain LIGA BBVA,3,0,2008-08-30 00:00:00,2008/2009
1,Spain,Spain LIGA BBVA,1,1,2008-08-31 00:00:00,2008/2009
2,Spain,Spain LIGA BBVA,2,1,2008-08-31 00:00:00,2008/2009
3,Spain,Spain LIGA BBVA,1,0,2008-08-31 00:00:00,2008/2009
4,Spain,Spain LIGA BBVA,1,1,2008-08-31 00:00:00,2008/2009
...,...,...,...,...,...,...
3035,Spain,Spain LIGA BBVA,2,1,2015-10-25 00:00:00,2015/2016
3036,Spain,Spain LIGA BBVA,2,0,2015-10-24 00:00:00,2015/2016
3037,Spain,Spain LIGA BBVA,3,0,2015-10-26 00:00:00,2015/2016
3038,Spain,Spain LIGA BBVA,1,1,2015-10-24 00:00:00,2015/2016


In [503]:
print('\n six) data grouped by season and league\n')
pd.read_sql("""
SELECT 
  Country.name AS country_name, League.name AS league_name, Match.season, 
  COUNT(DISTINCT home_team_api_id) AS distinct_home_teams,
  AVG(home_team_goal) AS average_home_goals,
  AVG(away_team_goal) AS average_away_goals,
  AVG(ABS(home_team_goal - away_team_goal)) AS average_goal_diff, 
  AVG(home_team_goal+away_team_goal) AS average_goal_sum,
  home_team_goal + home_team_goal AS goal_sum
  FROM Country 
    INNER JOIN League 
        ON Country.id = League.country_id 
        INNER JOIN Match ON League.id = Match.league_id
    WHERE Country.name='Spain' OR Country.name='Germany' OR
    Country.name='France' OR Country.name='Italy' OR Country.name='England' 
    GROUP BY Match.season, League.name;
""", conn1)


 six) data grouped by season and league



,country_name,league_name,season,distinct_home_teams,average_home_goals,average_away_goals,average_goal_diff,average_goal_sum,goal_sum
0,England,England Premier League,2008/2009,20,1.400000,1.078947,1.305263,2.478947,2
1,France,France Ligue 1,2008/2009,20,1.286842,0.971053,1.094737,2.257895,4
2,Germany,Germany 1. Bundesliga,2008/2009,18,1.699346,1.222222,1.431373,2.921569,4
3,Italy,Italy Serie A,2008/2009,20,1.521053,1.078947,1.284211,2.600000,2
4,Spain,Spain LIGA BBVA,2008/2009,20,1.660526,1.236842,1.381579,2.897368,6
5,England,England Premier League,2009/2010,20,1.697368,1.073684,1.471053,2.771053,2
6,France,France Ligue 1,2009/2010,20,1.389474,1.021053,1.294737,2.410526,0
7,Germany,Germany 1. Bundesliga,2009/2010,18,1.513072,1.316993,1.346405,2.830065,4
8,Italy,Italy Serie A,2009/2010,20,1.542105,1.068421,1.168421,2.610526,2
9,Spain,Spain LIGA BBVA,2009/2010,20,1.600000,1.113158,1.386842,2.713158,6


In [505]:
print('goal sum without grouping')
print(pd.read_sql("""
SELECT Country.name, League.name,
  (home_team_goal + home_team_goal) AS goal_sum
  FROM Country 
    INNER JOIN League 
        ON Country.id = League.country_id 
        INNER JOIN Match ON League.id = Match.league_id
    WHERE Country.name='Spain' OR Country.name='Germany' OR
    Country.name='France' OR Country.name='Italy' OR Country.name='England' 
    ;
""", conn1))

goal sum without grouping
          name                    name  goal_sum
0      England  England Premier League         2
1      England  England Premier League         2
2      England  England Premier League         0
3      England  England Premier League         4
4      England  England Premier League         8
...        ...                     ...       ...
14580    Spain         Spain LIGA BBVA         4
14581    Spain         Spain LIGA BBVA         4
14582    Spain         Spain LIGA BBVA         6
14583    Spain         Spain LIGA BBVA         2
14584    Spain         Spain LIGA BBVA         6

[14585 rows x 3 columns]
